# TITLE: Signal Modulation and Detection in an AWGN Channel with Laser Phase Noise and CPE
Authors: ...
Last update: 15/02/2023

## Initialization

In [1]:
# Python packages
import os
import sys
import math
import numpy as np
from tabulate import tabulate
from scipy.signal import welch
import matplotlib.pyplot as plt

# Set directory where PyDSP_core library is located:
PyDSP_core_dir = 'C:/Users/guiomar/Desktop/myGIT/libraries'
sys.path.append(PyDSP_core_dir)

# Custom packages
from PyDSP_core.TX.Tx_generateBits import Tx_generateBits, nextpow2
from PyDSP_core.TX.setSignalParams import setSignalParams
from PyDSP_core.TX.pulseShaper import pulseShaper
from PyDSP_core.TX.QAM_config import QAM_config
from PyDSP_core.TX.Tx_QAM import Tx_QAM
from PyDSP_core.TX.laserCW import laserCW
from PyDSP_core.TX.Tx_pilots import Tx_addPilots

from PyDSP_core.RX.setSNR import setSNR

from PyDSP_core.DSP.symDemapper import symDemapper
from PyDSP_core.DSP.rxDecision import BER_eval, EVM_eval, MI_eval
from PyDSP_core.DSP.LPF_apply import LPF_apply
from PyDSP_core.DSP.sync import pilotSymbols_rmv
from PyDSP_core.DSP.CPE import carrierPhaseEstimation

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (996262317.py, line 11)

## Input Parameters

In [ ]:
SIG = {
    'M': 16,            #QAM constellation size
    'symRate': 60e9,    #total symbol-rate of the signal
    'modulation': 'QAM',#modulation type [QAM/PAM]
    'rollOff': 0.1,     #roll-off factor
    'nPol': 1,          #number of polarizations
    'nSyms': pow(2, 18) #total number of simulated symbols
}
nSpS = 2                #number of samples per symbol
laserLW = 1e6           #laser linewidth [Hz]

pilotRate = 31/32
useCPE2 = True

SNR_dB = 17             #SNR in dB

## Set Transmitter Parameters

In [ ]:
# Signal Parameters:
TX = {'SIG': setSignalParams('symRate', SIG['symRate'], 'M', SIG['M'], 'nPol', SIG['nPol'],
                             'nBpS', math.log2(SIG['M']), 'nSyms', SIG['nSyms'],
                             'roll-off', SIG['rollOff'], 'modulation', SIG['modulation'])
}
# Modulation Parameters:
TX['QAM'] = QAM_config(TX['SIG'])
# Bit Parameters:
TX['BIT'] = {'source': 'randi',
             'seed': 1
}
# Pulse Shaping Filter Parameters:
TX['PS'] = {'type': 'RRC',
            'rollOff': TX['SIG']['rollOff'],
            'nTaps': 128
}
# DAC Parameters:
TX['DAC'] = {'RESAMP': {'sampRate':nSpS * TX['SIG']['symRate']}
}
#Laser Parameters:
TX['LASER'] = {
    'linewidth': laserLW,
}
# DSP Pilots Parameters:
TX['PILOTS'] = {'active':True,
                'rate': pilotRate,
#                'option': 'outerQPSK'
                'option': 'innerQPSK',
#                'option': 'meanQPSK',
#                'option': 'customQPSK',
                'scaleFactor': 1
}

## SET DSP Parameters

In [ ]:
#Matched-Filter Parameters:
DSP = {'MF':{'type': 'RRC',
             'rollOff': TX['SIG']['rollOff']}
}

#Carrier-Phase Estimation - 1st Stage:
DSP['CPE1'] = {
    'method': 'pilot-based',
    'nTaps': 5,
    'PILOTS': TX['PILOTS']
}

#Carrier-Phase Estimation - 2nd Stage:
DSP['CPE2'] = {
    'method': 'BPS',
    'nTaps': 22,
    'nTestPhases': 10,
    'angleInterval': np.pi/8
}

#Demapper:
DSP['DEMAPPER'] = {
    'normMethod': 'MMSE'
}

## Generate Tx Bits

In [ ]:
TX['BIT']['txBits'] = Tx_generateBits(SIG['nSyms'], TX['QAM']['M'], TX['QAM']['nPol'], TX['BIT'])

## Generate Transmitted Symbols

In [ ]:
S = {}
S['tx'], TX['BIT']['txSyms'] = Tx_QAM(TX['QAM'], TX['BIT']['txBits'])

# Plot Transmitted Constellation Symbols:
plt.scatter(S['tx'].real, S['tx'].imag, s=20)
plt.title('IQ Constellation of the Transmitted Signal')
plt.show()

## Add DSP Pilots

In [ ]:
if 'PILOTS' in TX  and TX['PILOTS']['active'] and TX['PILOTS']['rate'] < 1:
    S['tx'],TX['PILOTS']['pilotSequence'],TX['PILOTS']['idx'] = Tx_addPilots(S['tx'],TX['PILOTS'],TX['QAM']['IQmap'])
    TX['QAM']['meanConstPower'] = np.mean(abs(S['tx'])**2,1)
    TX['QAM']['maxConstPower'] = np.amax(abs(S['tx'])**2,1)

# Plot Pilot Symbols:
plt.scatter(S['tx'].real, S['tx'].imag, s=20)
plt.scatter(TX['PILOTS']['pilotSequence'].real,TX['PILOTS']['pilotSequence'].imag, color= 'red', s=20)
plt.title('IQ Constellation of the DSP Pilot Symbols')
plt.show()

## Pulse Shaping

In [ ]:
S['txSC'], TX['PS'] = pulseShaper(S['tx'], nSpS, TX['PS'])

#Plot Pulse Shaping Taps:
plt.plot(TX['PS']['W'])
plt.title('Pulse Shaping Taps')
plt.show()

# Plot Signal Spectrum:
# No matlab: pwelch(S.txSC(1,:),1e4,[],[],TX.DAC.RESAMP.sampRate,'centered')
#                  (x, window, noverlap, nfft, Fs, freqrange)
#
# window - If window is an integer, the signal is divided into segments of length window. The modified periodograms are computed using a Hamming window of length window.
# noverlap - If you do not specify noverlap, or specify noverlap as empty, the default number of overlapped samples is 50% of the window length.
# nfft - The default nfft is the greater of 256 or the next power of 2 greater than the length of the segments.

wind = np.hamming(pow(10,4))
nov = np.floor(pow(10,4)/2)
nff = max(256, pow(2,nextpow2(pow(10,4))))

f,Pxx = welch(S['txSC'][0],window = np.hamming(int(1e4)),noverlap=nov,nfft=nff,fs=TX['DAC']['RESAMP']['sampRate'],return_onesided=False ,detrend= False)
plt.plot(f,10*np.log10(Pxx))
plt.grid()
plt.show()

## Apply Laser

In [ ]:
Fs_SC = nSpS * SIG['symRate']
nSamples = S['tx'].shape[1] * nSpS
Scw, TX['LASER'] = laserCW(TX['LASER'],Fs_SC,nSamples)
if SIG['nPol'] == 2:
    Scw = Scw/np.sqrt(2)
for n in range(0, SIG['nPol']):
    S['txSC'][n,:] = Scw*S['txSC'][n,:]

# Plot Laser Phase Noise
plt.plot(TX['LASER']['phaseNoise'],linewidth=0.5)
plt.title('Transmitted Laser Phase Noise')
plt.show()

## Set SNR

In [ ]:
S['rx'] = setSNR(S['txSC'], SNR_dB, TX['DAC']['RESAMP']['sampRate'], SIG['symRate'])[0]

## Apply Matched Filter

In [ ]:
S['rx'] = LPF_apply(S['rx'],DSP['MF'],TX['DAC']['RESAMP']['sampRate'],SIG['symRate'])[0]

## Downsample to 1 Sample/Symbol

In [ ]:
S['rx_1sps'] = S['rx'][:,::nSpS]

## Plot Received Constellation

In [ ]:
for n in range(0, SIG['nPol']):
    plt.scatter(S['rx_1sps'][n].real, S['rx_1sps'][n].imag, s=0.1)
    plt.title('IQ Constellation of the Received Symbols: pol-'+str(n+1))
    plt.show()

## Apply 1st-Stage CPE: Pilot-Based

In [ ]:
S['rx_1sps'], DSP['CPE1'] = carrierPhaseEstimation(S['rx_1sps'],S['tx'],DSP['CPE1'])

## Apply 2nd-Stage CPE: Blind-Phase Search

In [ ]:
C = TX['QAM']['IQmap']
if useCPE2:
    S['rx_1sps'], DSP['CPE2'] = carrierPhaseEstimation(S['rx_1sps'],S['tx'],DSP['CPE2'],C)

## Plot Laser Phase Noise

In [ ]:
plt.plot(TX['LASER']['phaseNoise'][::nSpS],linewidth=0.5, label='Transmitted Laser Phase Noise' )
plt.plot(-DSP['CPE1']['phi'][0],linewidth=0.5, label='Estimated Laser Phase Noise')
plt.title('Estimated Laser Phase Noise at the Receiver')
plt.legend()
plt.show()

## Remove DSP pilots

In [ ]:
S['rx_1sps'], S['tx'] = pilotSymbols_rmv(S['rx_1sps'],S['tx'],DSP['CPE1']['PILOTS']['rate'],DSP['CPE1']['PILOTS']['pilotSequence'])[0:2]

## Apply Symbol Demapper

In [ ]:
C = TX['QAM']['IQmap']
DSP['DEMAPPER'],S['tx'] = symDemapper(S['rx_1sps'],S['tx'],C,DSP['DEMAPPER'])

## Performance Analysis

In [ ]:
BER = np.zeros(SIG['nPol'])
errPos = []
EVM = np.zeros(SIG['nPol'])
MI = np.zeros(SIG['nPol'])
for n in range(0, SIG['nPol']):
    BER[n], errPos_aux = BER_eval(DSP['DEMAPPER']['txBits'][n],DSP['DEMAPPER']['rxBits'][n])
    errPos.append(errPos_aux)
    EVM[n] = EVM_eval(S['rx_1sps'][n],S['tx'][n])
    MI[n] = MI_eval(S['rx_1sps'][n],S['tx'][n],DSP['DEMAPPER']['C'][:,n],DSP['DEMAPPER']['N0'][n])

## Plot Constellation after Demapping

In [ ]:
for n in range(0, SIG['nPol']):
    plt.scatter(S['rx_1sps'][n].real, S['rx_1sps'][n].imag, s=0.1)
    plt.scatter(DSP['DEMAPPER']['C'][:,n].real, DSP['DEMAPPER']['C'][:,n].imag,s=50, marker='o', edgecolor='k', facecolor=(0,1,0,0))
    plt.title('pol-'+str(n+1))
    plt.show()

## Print Results

In [ ]:
parameters = ['BER', 'EVM', 'MI']
columns = np.asarray([BER,EVM,MI])
header = [f"Polarization-{i}" for i in range(1,SIG['nPol'] + 1)]

print(tabulate(columns,headers=header,tablefmt= 'fancy_grid',showindex=parameters,numalign='right'))